Command Line Interface Script to Serving Model Tensorflow.

1. Create Dockerfile
2. Build Images
   ```
   docker build -t tf_intent_serving .
   ```
3. Start Container
    ```
    docker run -p 8501:8501 --name intent_classification -d tf_intent_serving
    ```

In [13]:
# Load libraries
import json
import base64
import requests
import pandas as pd 
import tensorflow as tf

In [5]:
# Read dataset
data = pd.read_csv("dataset/atis_intents_preprocessed.csv")

In [6]:
# Overview dataset
data.sample(5)

,message,is_flight_intent
4298,i want to fly denver to pittsburgh,1
2415,please list the flights from baltimore to san...,1
463,are there any flights from atlanta to denver,1
3785,what is the latest flight from salt lake city...,1
3279,what are the flights from san francisco to de...,1


In [41]:
# Sampling 1 set
test_sample_data = data.sample(1)
label_actual =  test_sample_data['is_flight_intent'].values[0]
test_sample_data = test_sample_data['message'].values[0]
test_sample_data

' san francisco to pittsburgh friday'

In [42]:
# Create function to prepare data for inference
def prepare_json(text: str):
    feature_spec = {"message": tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(text, 'utf-8')]
        )
    )}
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    result = [{
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }]
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    }, indent=4)

In [43]:
# Overview 
json_data = prepare_json(text=test_sample_data)
print(json_data)

{
    "signature_name": "serving_default",
    "instances": [
        {
            "examples": {
                "b64": "CjQKMgoHbWVzc2FnZRInCiUKIyBzYW4gZnJhbmNpc2NvIHRvIHBpdHRzYnVyZ2ggZnJpZGF5"
            }
        }
    ]
}


In [44]:
# Define URL Endpoint gRPC
endpoint = "http://localhost:8501/v1/models/intent-flight-classification:predict"

In [45]:
%%time
# Execute service
response = requests.post(
    endpoint, data=json_data
)

CPU times: total: 0 ns
Wall time: 22 ms


In [46]:
# Overview response
print(response.content.decode())

{
    "predictions": [[0.999740362]
    ]
}


In [47]:
# Overview actual label
label_actual

1